# 웹 스크래핑 프로그램 만들어보자

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import math, time

options = Options()
options.add_argument('--window-size=974,1047')
options.add_argument('--window-position=-7,0')
options.add_experimental_option("detach", True)

In [2]:
search = input('검색어:')
cnt = int(input('크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)  # 크롤링 할 전체 페이지 수 

검색어: 제주도
크롤링 할 건수는 몇건입니까?:  10


In [3]:
URL = 'https://korean.visitkorea.or.kr/search/search_list.do?keyword='+search
driver = webdriver.Chrome(options=options)
driver.get(URL)
time.sleep(3)

In [4]:
# 여행기사 더보기 클릭
# driver.find_element(By.CSS_SELECTOR, ".more_view").click() 중복주의
driver.find_element(By.CSS_SELECTOR, "#s_recommend > .more_view > a").click()

In [5]:
# 제목 추출
# 왜 길이가 34일까?
result = driver.find_elements(By.CSS_SELECTOR,'.tit a')
len(result)

34

In [6]:
for i in result:
    print(i.text)

























해변산책부터 레이싱까지, 제주 반려동물 동반여행 추천 코스
제주도가 품은 신비의 화산 숲, 환상숲곶자왈공원
한국관광공사 추천 2월 걷기여행길, 이른 봄을 맞이하는 제주도 걷기길
제주도의 숨겨진 재미를 찾는
<놀멍, 배우멍 제주마을체험 - 1일형 체험학습>
제주도 조천읍 버킷리스트 여행지 작성하기
김창열의 회귀 철학을 건축으로 표현한 미술관, 제주도립김창열미술관
[국내 트레킹 추천]
제주 올레길 걷기 여행,
준비부터 코스 선택까지 꿀팁 총정리!
4월 제주도 가볼 만한, 가파도 청보리 축제
제주도 유채꽃 명소 추천, 봄에 떠나는 인생샷 여행
소소한 '제주여행' 꿀팁! 〈겨울편〉


In [7]:
# 더 정확하게 찾아주자
result = driver.find_elements(By.CSS_SELECTOR,'#search_result .tit>a')
len(result)

10

In [8]:
# 한페이지내 콘텐츠별 상세페이지 정보를 추출해보자
# 페이지 로딩시간 고려해서 넉넉히 2~3초 간격을 주자
from bs4 import BeautifulSoup
contents_no = 0

for item in result:
    contents_no += 1
    item.click()
    time.sleep(3)  
    
    print(f'======= [ {contents_no} ]  =======')
    html = driver.page_source

    soup = BeautifulSoup(html, 'lxml')
    title = soup.find(id='topTitle')
    print(title.text)

    driver.back()
    time.sleep(3)

print('==== 완료 ====')

======= [ 1 ]  =======
해변산책부터 레이싱까지, 제주 반려동물 동반여행 추천 코스


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=126.0.6478.126); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF79402EEA2+31554]
	(No symbol) [0x00007FF793FA7ED9]
	(No symbol) [0x00007FF793E6872A]
	(No symbol) [0x00007FF793E7ADBD]
	(No symbol) [0x00007FF793E7B84A]
	(No symbol) [0x00007FF793E6F5BB]
	(No symbol) [0x00007FF793E6D326]
	(No symbol) [0x00007FF793E70EB1]
	(No symbol) [0x00007FF793E70F50]
	(No symbol) [0x00007FF793EBA48C]
	(No symbol) [0x00007FF793EAC006]
	(No symbol) [0x00007FF793EDD02A]
	(No symbol) [0x00007FF793EABA76]
	(No symbol) [0x00007FF793EDD240]
	(No symbol) [0x00007FF793EFC977]
	(No symbol) [0x00007FF793EDCDD3]
	(No symbol) [0x00007FF793EAA33B]
	(No symbol) [0x00007FF793EAAED1]
	GetHandleVerifier [0x00007FF794338B1D+3217341]
	GetHandleVerifier [0x00007FF794

In [9]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import math, time, os, pandas as pd, urllib.request

options = Options()
options.add_argument('--window-size=974,1047')
options.add_argument('--window-position=-7,0')
options.add_experimental_option("detach", True)

search = input('검색어:')
cnt = int(input('스크래핑 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)  # 크롤링 할 전체 페이지 수 

now = time.localtime()
date_format = '%04d%02d%02d'%(now.tm_year, now.tm_mon, now.tm_mday)
f_dir = f'{os.getcwd()}\\{search}여행기사_{cnt}건_{date_format}'
os.makedirs(f_dir)

URL = 'https://korean.visitkorea.or.kr/search/search_list.do?keyword='+search
driver = webdriver.Chrome(options=options)
driver.get(URL)
time.sleep(2)
# 여행기사 더보기 클릭
driver.find_element(By.CSS_SELECTOR, "#s_recommend > .more_view > a").click()
time.sleep(2)

title_list = []
contents_list = []
img_url_list = []

def page_work():
    result = driver.find_elements(By.CSS_SELECTOR,'#search_result .tit>a')
    global contents_no, cnt
    global title_list, contents_list, img_url_list

    for item in result:
        contents_no += 1

        if contents_no <= cnt :    
            print(f'[콘텐츠 {contents_no}]')  
            item.send_keys(Keys.ENTER) # .click()은 에러 잘남

            time.sleep(2)

            # 이미지 추출을 위해 미리 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

            html = driver.page_source
            html_dom = BeautifulSoup(html, 'lxml')

            title = html_dom.find(id='topTitle')
            title_list.append(title.text)
            print(title.text)

            img_tag_list = html_dom.select('.img_typeBox img')
            img_url_list = [item['src'] for item in img_tag_list]

            contents = driver.find_elements(By.CLASS_NAME, 'txt_p')
            contents_merge = ' '.join([item.text for item in contents])        
            contents_list.append(contents_merge)           

            driver.back()
            time.sleep(2)     


def file_export():

    DF = pd.DataFrame({"제목":title_list, "내용":contents_list})
    filename = f'{search}여행기사_{cnt}건_{date_format}.xlsx'
    DF.to_excel(f_dir+'\\'+filename)
    print(f'====== {page_no} 페이지 {filename} 파일 저장 완료 ======')


    no = 0
    for src in img_url_list:
        # 다운로드  (주소, 파일이름)
        urllib.request.urlretrieve(src, f'{f_dir}\\{page_no}_{no}.jpg')
        no += 1
    print(f'====== {page_no} 페이지 {f_dir} 디렉토리 이미지 저장 완료 ======')


contents_no = 0
today = time.localtime()
print('스크래핑 프로그램 실행')

for page_no in range(1, page_cnt+1):    
    print(f'====== {page_no} 페이지 스크래핑 시작 ======')
    page_work()
    print(f'====== {page_no} 페이지 스크래핑 작업중 ======')
    file_export()
    print(f'====== {page_no} 페이지 스크래핑 완료 ======')
    if page_no < page_cnt:
        driver.find_element(By.XPATH, f'/html/body/div[3]/div/div[1]/div[14]/a[{page_no+1}]').click()
        time.sleep(2)

print('스크래핑 프로그램 종료')
driver.close()

검색어: 제주도
스크래핑 할 건수는 몇건입니까?:  15


스크래핑 프로그램 실행
====== 1 페이지 스크래핑 시작 ======
[콘텐츠 1]
해변산책부터 레이싱까지, 제주 반려동물 동반여행 추천 코스
[콘텐츠 2]


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=126.0.6478.126); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF79402EEA2+31554]
	(No symbol) [0x00007FF793FA7ED9]
	(No symbol) [0x00007FF793E6872A]
	(No symbol) [0x00007FF793E7ADBD]
	(No symbol) [0x00007FF793E7B84A]
	(No symbol) [0x00007FF793E6F5BB]
	(No symbol) [0x00007FF793E6D326]
	(No symbol) [0x00007FF793E70EB1]
	(No symbol) [0x00007FF793E70F50]
	(No symbol) [0x00007FF793EB9192]
	(No symbol) [0x00007FF793EB9272]
	(No symbol) [0x00007FF793EAF119]
	(No symbol) [0x00007FF793EDD02A]
	(No symbol) [0x00007FF793EABA76]
	(No symbol) [0x00007FF793EDD240]
	(No symbol) [0x00007FF793EFC977]
	(No symbol) [0x00007FF793EDCDD3]
	(No symbol) [0x00007FF793EAA33B]
	(No symbol) [0x00007FF793EAAED1]
	GetHandleVerifier [0x00007FF794338B1D+3217341